<a href="https://colab.research.google.com/github/avirus2021/Awesome-explainable-AI/blob/master/CNN_Botanical_Research_proj_Amaresh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

#Botanical Research

• DOMAIN: Botanical Research

• CONTEXT: University X is currently undergoing some research involving understanding the characteristics of plant and plant seedlings at various stages of growth. They already have have invested on curating sample
images. They require an automation which can create a classifier capable of determining a plant's species from a photo.

• DATA DESCRIPTION: The dataset comprises of images from 12 plant species. Source: https://www.kaggle.com/c/plant-seedlings-classification/data

• PROJECT OBJECTIVE: University’s management require an automation which can create a classifier capable of determining a plant's species from a photo.


In [2]:
project_path = "/content/drive/MyDrive/Colab Notebooks/PROJECT/COMPUTER VISION"

In [3]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Colab Notebooks/PROJECT/COMPUTER VISION/test.zip', 'r') as z:
  z.extractall()

In [4]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Colab Notebooks/PROJECT/COMPUTER VISION/train.zip', 'r') as z:
  z.extractall()

In [5]:
import pandas as pd
import os

In [6]:
species_class = os.listdir('/content/train')

In [7]:
BASE_DATA_FOLDER = '/content'
TRAIN_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")

In [8]:
species_class

['Sugar beet',
 'Common wheat',
 'Fat Hen',
 'Common Chickweed',
 'Maize',
 'Loose Silky-bent',
 'Black-grass',
 'Shepherds Purse',
 'Scentless Mayweed',
 'Small-flowered Cranesbill',
 'Charlock',
 'Cleavers']

In [9]:
from glob import glob
import cv2
images_per_class = {}
for class_folder_name in os.listdir(TRAIN_DATA_FOLDER):
    class_folder_path = os.path.join(TRAIN_DATA_FOLDER, class_folder_name)
    class_label = class_folder_name
    images_per_class[class_label] = []
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
        images_per_class[class_label].append(image_bgr)

In [10]:
for key,value in images_per_class.items():
    print("{0} = {1}".format(key, len(value)))

Sugar beet = 385
Common wheat = 221
Fat Hen = 475
Common Chickweed = 611
Maize = 221
Loose Silky-bent = 671
Black-grass = 263
Shepherds Purse = 231
Scentless Mayweed = 516
Small-flowered Cranesbill = 496
Charlock = 390
Cleavers = 287


Read unique id given to each image as filename

In [11]:
train = []

for species_num, species in enumerate(species_class):
    for file in os.listdir(os.path.join(TRAIN_DATA_FOLDER, species)):
        train.append(['/content/train/{}/{}'.format(species, file), species_num, species])
        
train = pd.DataFrame(train, columns=['file', 'species_num', 'species'])

print('Training Data: ',train.shape)

Training Data:  (4767, 3)


In [12]:
train.head()

,file,species_num,species
0,/content/train/Sugar beet/da2912b2f.png,0,Sugar beet
1,/content/train/Sugar beet/99926b786.png,0,Sugar beet
2,/content/train/Sugar beet/a8ff2c49e.png,0,Sugar beet
3,/content/train/Sugar beet/84d42e202.png,0,Sugar beet
4,/content/train/Sugar beet/9f70ba1fb.png,0,Sugar beet



One-hot encodings of species

Encode train['species'] column

In [13]:
import numpy as np
targets = pd.Series(train['species'])
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [14]:
one_hot_labels[500]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Define img_rows, img_cols, and num_channel variables

In [15]:
img_rows=256
img_cols=256
num_channel=1


Get features

In [16]:
# from tqdm import tqdm
# import cv2
# x_feature = []
# y_feature = []

# i = 0 # initialisation
# for f, img in tqdm(train.values): # f for format ,png
#  train_img = cv2.imread('./train/{}.jpg'.format(f),0)
#  label = one_hot_labels[i]
# train_img_resize = cv2.resize(train_img, (256, 256)) 
# x_feature.append(train_img_resize)
# y_feature.append(label)
# i += 1



In [17]:
# X = []
#count = 0

#path = glob.glob('/content/train')
#for img in os.listdir('/content/train'):
    #image = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
    #try:
       # image = cv2.resize(image, (img_rows, img_cols), interpolation=cv2.INTER_AREA)
      #  print(image.shape)
    #except:
    #    break
   # X.append([image])
   # count = count +1
#print(count)

In [18]:
# import glob
# import cv2
# folders = glob.glob('/content/train')
# imagenames_list = []
# for folder in folders:
#   for f in glob.glob(folder + '/*.png'):
#    imagenames_list.append(f)

# read_images = []
# for image in imagenames_list: 
#   read_images.append(cv2.imread(image, cv2.IMREAD_GRAYSCALE))

In [19]:
#x_train_data = np.array(imagenames_list, np.float32) / 255.   # /= 255 for normolisation
#print (x_train_data.shape)

In [20]:
# #import necessary packages 
# import cv2 
# import os 
# import glob 
# import matplotlib.pyplot as plt 
# %matplotlib inline 
 
# #Set the path where images are stored 
# img_dir = "/content/train" # Enter Directory of all images  
# data_path = os.path.join(img_dir,'*.png') 
# files = glob.glob(data_path) 
# data = [] 
# for f1 in files: 
#     img = cv2.imread(f1) 
#     data.append(img) 
#     plt.figure() 
#     plt.imshow(img)

In [21]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [27]:
  x_train = []

  for i in range(len(train)):
      img = cv2.imread(train['file'][i])
      img = cv2.resize(img,dsize=(256,256))
#     img_stack = segment_plant(img)
#     img_stack = sharpen_image(img_stack)
#     img_stack = cv2.cvtColor( img_stack, cv2.COLOR_RGB2GRAY )
#     img_stack = np.reshape(img_stack,(256,256,1))
      x_train.append(np.array(img))

  x_train = np.array(x_train)

In [28]:
x_train_data = np.array(x_train, np.float32) / 255.   # /= 255 for normolisation
print (x_train_data.shape)

(4767, 256, 256, 3)


In [29]:
y_train_data = np.array(one_hot_labels)
y_train_data.shape

(4767, 12)

Split the training and validation data here

In [30]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_data, y_train_data, test_size=0.3, random_state=2)

In [ ]:
print (x_train.shape)
print (x_val.shape)